In [1]:
%load_ext autoreload
%autoreload 2
import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
from livenet.backend.core import Context
import random
import importlib
import onnx
import livenet
device = "cuda"
# device = "cpu"
torch.set_default_device(device)
from ai_libs.simple_log import LOG



In [7]:
# transform = transforms.Compose(
#     [transforms.ToTensor()
#         ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#      ])
# train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
#                                   download=True, transform=transform)
# test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
#                                   download=True, transform=transform)
#
#
# def get_whole_data(dataset):
#     loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))
#     data = next(iter(loader))
#     data[0] = data[0].to(device)
#     data[1] = data[1].to(device)
#     return data
#
# test_whole_data = get_whole_data(test)
# train_whole_data = get_whole_data(train)

# test_x, test_y = test_whole_data
# train_x, train_y = train_whole_data

test_x, test_y = livenet.datasets.get_cifar10_test()
train_x, train_y = livenet.datasets.get_cifar10_train(augment=True)

test_x = test_x.to(device)
test_y = test_y.to(device)
train_x = train_x.to(device)
train_y = train_y.to(device)



In [46]:


def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(0)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.context = Context(self)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.av_pool = nn.AvgPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 8, 3)
        # self.conv2 = nn.Conv2d(8, 8, 3,padding="same")

        self.conv2b = nn.Conv2d(8, 16, 3)
        self.conv3b = nn.Conv2d(16, 16, 3, groups=16)
        self.conv4b = nn.Conv2d(16, 32, 1)
        self.conv5b = nn.Conv2d(32, 32, 3, groups=32)
        self.conv6b = nn.Conv2d(32, 64, 1)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1b = nn.Linear(64, 10)

        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(16)
        self.bn4 = nn.BatchNorm2d(32)
        self.bn5 = nn.BatchNorm2d(32)
        self.bn6 = nn.BatchNorm2d(64)

        self.fc = nn.Linear(8*7*7,10)
        self._alpha = 0.0001

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        # x = self.max_pool(x)
        # x = self.av_pool(x)

        # x = F.relu(self.conv2(x))
        x = F.relu(self.bn2(self.conv2b(x)))
        x = self.av_pool(x)
        x = F.relu(self.bn3(self.conv3b(x)))
        x = self.av_pool(x)
        x = F.relu(self.bn4(self.conv4b(x)))
        x = F.relu(self.bn5(self.conv5b(x)))
        x = self.av_pool(x)
        x = F.relu(self.bn6(self.conv6b(x)))
        x = self.av_pool(x)
        # x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        x=self.fc1b(x)
        return x

    def internal_loss(self):
        loss = torch.tensor(0.)
        for param in self.parameters():
            if len(param.data.shape) > 1:
                loss += self._alpha * torch.sum(torch.abs(param)) / param.data.numel()
        return loss


network = Net()
torchsummary.summary(network, (3, 32, 32), device=device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 30, 30]             224
       BatchNorm2d-2            [-1, 8, 30, 30]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         AvgPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 12, 12]             160
       BatchNorm2d-7           [-1, 16, 12, 12]              32
         AvgPool2d-8             [-1, 16, 6, 6]               0
            Conv2d-9             [-1, 32, 6, 6]             544
      BatchNorm2d-10             [-1, 32, 6, 6]              64
           Conv2d-11             [-1, 32, 4, 4]             320
      BatchNorm2d-12             [-1, 32, 4, 4]              64
        AvgPool2d-13             [-1, 32, 2, 2]               0
           Conv2d-14             [-1, 6

In [47]:
batch_size = 256
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=len(train_x) // batch_size)
trainer.adaptive_lr = True
optimizer.learning_rate = 0.01



Torch


In [52]:
#network.learning_rate = 0.0005
# optimizer.learning_rate /= 2
trainer.step(20000)


Iˈ0.000 20279 1.004+0.001reg params=30 lr=0.01000 <string>:34
Iˈ3.181 20669 1.002+0.001reg params=30 lr=0.01000 <string>:34
Iˈ6.041 21059 1.001+0.001reg params=30 lr=0.01000 <string>:34
Iˈ9.265 21449 0.997+0.001reg params=30 lr=0.01000 <string>:34
Iˈ12.483 21839 0.999+0.001reg params=30 lr=0.01000 <string>:34
Iˈ15.622 22229 0.995+0.001reg params=30 lr=0.01000 <string>:34
Iˈ18.761 22619 0.996+0.001reg params=30 lr=0.01000 <string>:34
Iˈ21.972 23009 0.992+0.001reg params=30 lr=0.01000 <string>:34
Iˈ25.150 23399 0.993+0.001reg params=30 lr=0.01000 <string>:34
Iˈ28.110 23789 0.991+0.001reg params=30 lr=0.01000 <string>:34
Iˈ30.935 24179 0.994+0.001reg params=30 lr=0.01000 <string>:34
Iˈ33.783 24569 0.987+0.001reg params=30 lr=0.01000 <string>:34
Iˈ36.634 24959 0.991+0.001reg params=30 lr=0.01000 <string>:34
Iˈ39.414 25349 0.989+0.001reg params=30 lr=0.01000 <string>:34
Iˈ42.301 25739 0.985+0.001reg params=30 lr=0.01000 <string>:34
Iˈ45.076 26129 0.986+0.001reg params=30 lr=0.01000 <string>

In [54]:
import gc
gc.collect()
torch.cuda.empty_cache()

def calc_accuracy(predictions, labels):
    _, predicted = torch.max(predictions.data, 1)
    labels = labels.cpu().numpy()
    labels = np.squeeze(labels, 1)
    predicted = predicted.cpu().numpy()
    correct = np.sum(predicted == labels)
    total = len(labels)
    accuracy = correct / total
    return accuracy

def _infer_in_chunks(network, data):
    chunk_size = 256
    n = len(data)
    preds = []
    for i in range( (n + chunk_size - 1) // chunk_size):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, n)
        with torch.no_grad():
            pred = network(data[start:end])
            preds.append(pred)
    result = torch.concatenate(preds, dim=0)
    return result


def get_loss(trainer, x, y):
    with torch.no_grad():
        pred = _infer_in_chunks(trainer.network, x)
        loss = trainer.criterion(pred, y)
        loss = loss.cpu().item()
        return loss

train_loss = get_loss(trainer, train_x, train_y)
test_loss = get_loss(trainer, test_x, test_y)
LOG(f"loss: train: {train_loss:.3f} test: {test_loss:.3f}")

with torch.no_grad():
    train_outputs = _infer_in_chunks(network, train_x)
    test_outputs = _infer_in_chunks(network, test_x)

train_accuracy = calc_accuracy(train_outputs, train_y)
test_accuracy = calc_accuracy(test_outputs, test_y)
LOG(f"accuracy, train: {100 * train_accuracy:.1f}% test: {100 * test_accuracy:.1f}%")

Iˈ0.000 loss: train: 0.905 test: 1.079 
Iˈ0.425 accuracy, train: 77.9% test: 73.9% 


In [13]:
def test_func():
    return livenet.gen_utils.batch_iterator(*test_whole_data, batch_size=256, only_one_epoch=True)

def train_func():
    return livenet.gen_utils.batch_iterator(*train_whole_data, batch_size=256, only_one_epoch=True)
import time

net._alpha = 0.0001

def criterion(input, target):
    return nn.functional.cross_entropy(input, target) / math.log(2)

# optimizer = lib.optimizer.MyOptimizer(net.parameters(), lr=0.01)
t0 = time.time()
for epoch in range(50):
    print(f"{time.time() - t0:.3f} sec")
    t0 = time.time()

    running_loss = 0.0
    for i, data in enumerate(train_func(), 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss += net.reg_loss_func()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        n_observe = 4 * 195

        # print statistics
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i:5d}] running loss: {running_loss / i:.2f}')
    running_loss = 0.0
    calc_stats(net, train_whole_data)
    calc_stats(net, test_whole_data)
    #lr schedule
    if True:
        observed = np.array(losses[-n_observe:])
        av1 = np.average(observed[:len(observed) // 2])
        av2 = np.average(observed[len(observed) // 2:])
        print(f"av1={av1:.4f} av2={av2:.4f}")
        slope, pvalue = livenet.stat_utils.get_slope_and_pvalue(losses[-n_observe:])
        print(f"slope={slope:.1e} pvalue={pvalue:.1e} lr={optimizer.param_groups[0]['lr']}")
        if slope >= 0.0:
            optimizer.param_groups[0]["lr"] /= 1.4
            print(f"reduced lr to {optimizer.param_groups[0]['lr']:.2e}")

print('Finished Training')


NameError: name 'net' is not defined